# LangChain 및 필요한 라이브러리 설치

In [ ]:
!pip install langchain unstructured[all-docs] pydantic lxml langchain_openai langchain-community chromadb

In [ ]:
!apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.4 [186 kB]
Fetched 186 kB in 2s (99.0 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 123589 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.4_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.4) ...
Setting up poppler-utils (22.02.0-2ubuntu0.4) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
!apt install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 3s (1,535 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 123619 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [ ]:
!pip install pytesseract

In [ ]:
%cd /content/drive/MyDrive/miraeasset/code

[Errno 2] No such file or directory: '/content/drive/MyDrive/miraeasset/code'
/content


In [ ]:
import os
from typing import Any
from pydantic import BaseModel
from unstructured.partition.pdf import partition_pdf

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import SimpleSequentialChain, TransformChain, LLMChain

import base64
import json
import http.client
import time

import concurrent.futures
from embedding import HyperClovaEmbedding, HyperClovaEmbeddings # custom: embedding.py

from langchain.embeddings.base import Embeddings
from langchain.prompts import PromptTemplate

import uuid
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document

from langchain.schema.runnable import RunnablePassthrough
from langchain_core.prompts import MessagesPlaceholder
from operator import itemgetter

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

import pickle
import requests

# Data Loading

- PDF 표(Tables)와 텍스트(text) 분할
- Unstructured의 partition_pdf를 사용하여 레이아웃 모델을 이용해 PDF 문서를 세분화

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
class PDFPartitionConfig(BaseModel):
    extract_images_in_pdf: bool = True
    infer_table_structure: bool = True
    chunking_strategy: str = "by_title"
    max_characters: int = 4000
    new_after_n_chars: int = 3800
    combine_text_under_n_chars: int = 2000
    image_output_dir_path: str

def process_pdfs_in_directory(directory: str, config: PDFPartitionConfig) -> Any:
    raw_pdf_elements = []
    for filename in os.listdir(directory):
        if filename.endswith(".pdf"):
            file_path = os.path.join(directory, filename)
            pdf_elements = partition_pdf(
                filename=file_path,
                extract_images_in_pdf=config.extract_images_in_pdf,
                infer_table_structure=config.infer_table_structure,
                chunking_strategy=config.chunking_strategy,
                max_characters=config.max_characters,
                new_after_n_chars=config.new_after_n_chars,
                combine_text_under_n_chars=config.combine_text_under_n_chars,
                image_output_dir_path=config.image_output_dir_path
            )
            raw_pdf_elements.append((filename, pdf_elements))
    return raw_pdf_elements


# db_law

In [ ]:
path = "/content/drive/MyDrive/miraeasset/data/law"
config = PDFPartitionConfig(image_output_dir_path=path)

In [ ]:
raw_pdf_elements = process_pdfs_in_directory(path, config)

yolox_l0.05.onnx:   0%|          | 0.00/217M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
len(raw_pdf_elements)

1

In [ ]:
all_pdf_elements = []
for _, elements in raw_pdf_elements:
    all_pdf_elements.extend(elements)

In [ ]:
# Create a dictionary to store counts of each type
category_counts = {}

for element in all_pdf_elements:
    category = str(type(element))
    if category in category_counts:
        category_counts[category] += 1
    else:
        category_counts[category] = 1

# Unique_categories will have unique elements
unique_categories = set(category_counts.keys())
category_counts

{"<class 'unstructured.documents.elements.CompositeElement'>": 163,
 "<class 'unstructured.documents.elements.Table'>": 162}

In [ ]:
class Element(BaseModel):
    type: str
    text: Any


# Categorize by type
categorized_elements = []
for element in all_pdf_elements:
    if "unstructured.documents.elements.Table" in str(type(element)):
        categorized_elements.append(Element(type="table", text=str(element)))
    elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
        categorized_elements.append(Element(type="text", text=str(element)))

# Tables
table_elements = [e for e in categorized_elements if e.type == "table"]
print(len(table_elements))

# Text
text_elements = [e for e in categorized_elements if e.type == "text"]
print(len(text_elements))

162
163


## Summaries

In [ ]:
class CompletionExecutor:
    def __init__(self, host, api_key, api_key_primary_val, request_id, retry_attempts=10, initial_retry_delay=5):
        self._host = host
        self._api_key = api_key
        self._api_key_primary_val = api_key_primary_val
        self._request_id = request_id
        self.retry_attempts = retry_attempts
        self.initial_retry_delay = initial_retry_delay

    def _send_request(self, completion_request):
        headers = {
            'Content-Type': 'application/json; charset=utf-8',
            'X-NCP-CLOVASTUDIO-API-KEY': self._api_key,
            'X-NCP-APIGW-API-KEY': self._api_key_primary_val,
            'X-NCP-CLOVASTUDIO-REQUEST-ID': self._request_id
        }

        retry_delay = self.initial_retry_delay
        for attempt in range(self.retry_attempts):
            conn = http.client.HTTPSConnection(self._host)
            conn.request('POST', '/testapp/v1/api-tools/summarization/v2/177a87b6436e4d22815fded90f5d2dce', json.dumps(completion_request), headers)
            response = conn.getresponse()
            if response.status == 429:
                # 요청이 너무 많음 (Rate limit exceeded)
                print(f"Rate limit exceeded. Retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)
                retry_delay *= 2  # 지수 백오프 (Exponential backoff)
                continue
            elif response.status != 200:
                conn.close()
                raise ValueError(f"HTTP error: {response.status} - {response.reason}")
            result = json.loads(response.read().decode(encoding='utf-8'))
            conn.close()
            if result['status']['code'] == '20000':
                return result
            else:
                raise ValueError('Error in API response: ' + result.get('status', {}).get('message', 'Unknown error'))

        raise ValueError('Max retry attempts exceeded')

    def execute(self, completion_request):
        try:
            res = self._send_request(completion_request)
            return res['result']['text']
        except ValueError as e:
            print(f"Error: {e}")
            return 'Error'


In [ ]:
# Define a function to use the HyperCLOVA API for summarization
def hyperclova_summarize(text):
    completion_executor = CompletionExecutor(
        host='clovastudio.apigw.ntruss.com',
        api_key='your_api_key_here',  # Replace with your actual API key
        api_key_primary_val='your_primary_api_key_here',  # Replace with your actual primary API key
        request_id='your_request_id_here' # Replace with your actual request ID
    )

    request_data = {
        "texts": [text],
        "segMinSize": 300,
        "includeAiFilters": True,
        "autoSentenceSplitter": True,
        "segCount": -1,
        "segMaxSize": 1000
    }

    response_text = completion_executor.execute(request_data)
    return response_text

In [ ]:
# Process texts concurrently and maintain order
def process_texts_concurrently(texts):
    results = [None] * len(texts)  # Placeholder for results
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Create a dictionary to track which future corresponds to which index
        future_to_index = {executor.submit(hyperclova_summarize, text): i for i, text in enumerate(texts)}
        for future in concurrent.futures.as_completed(future_to_index):
            index = future_to_index[future]
            try:
                result = future.result()
                results[index] = result
            except Exception as exc:
                print(f'Text at index {index} generated an exception: {exc}')
                results[index] = None  # Or handle the exception as needed
    return results

In [ ]:
tables = [i.text for i in table_elements]
table_summaries = process_texts_concurrently(tables)

Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 40 seconds...
Rate limit exceeded. Retrying in 40 seconds...
Rate limit exceeded. Retrying in 40 seconds...
Rate limit exceeded

In [ ]:
len(table_summaries)

162

In [ ]:
tables[5:8] # 확인용

['현 행 개정안 해외현지법인 파견 임직원 인건비에 대한 손금인 손금인정 범위 및 요건 개선 정 요건 ○ 중소ㆍ중견기업 ○ 직ㆍ간접적으로 100% 출자한 해외현지법인 ○ 내국법인 ○ (좌 동) ○ 내국법인이 지급한 인건비가 내국법인 및 해외 ○ (좌 동) 출자법인이 지급한 인건비 합계액의 50% 미만 인 경우 <추 가> ○ 해외현지법인 파견 임직원의 근로소득세를 내국 법인이 원천징수하여 납부하는 경우 * 소득세법상 거주자에 해당하고 소득세로 납부한 경 우에 한정',
 '현 행 개정안 손금 및 필요경비의 범위 근로자 출산·양육 지원금을 손금 및 필요경비에 추가 ○ 판매한 상품·제품에 대한 원료의 매입가액· 부대비용 ○ (좌 동) ○ 인건비 <추 가> ○ 근로자에게 지급하는 출산·양육 지원금 * 근로자에게 공통 적용되는 지급기준에 따른 것에 한함',
 '현 행 개정안 세금과 공과금 손금불산입 법령상 의무 불이행에 따른 공과금에서 포함되는 ○ 법령상 의무 불이행에 대한 제재로 부과되는 범위 명확화 공과금 <추 가> － 장애인고용부담금']

In [ ]:
table_summaries[5:8] # 확인용

['- 해외현지법인 파견 임직원 인건비에 대한 손금인정 범위 및 요건을 개선함',
 '- 근로자 출산·양육 지원금을 손금 및 필요경비에 추가함',
 '- 세금과 공과금 손금불산입 법령상 의무 불이행에 따른 공과금에서 포함되는 법령상 의무 불이행에 대한 제재로 부과되는 범위 명확화 공과금 추가함']

In [ ]:
texts = [i.text for i in text_elements]
text_summaries = process_texts_concurrently(texts)

Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 40 seconds...
Rate limit exceeded. Retrying in 40 seconds...
Rate limit exceeded. Retrying in 40 seconds...
Rate limit exceeded

In [ ]:
len(text_summaries)

163

## Add to vectorstore

In [ ]:
# HyperClovaEmbedding 인스턴스 생성
hyper_clova = HyperClovaEmbedding(
    host='clovastudio.apigw.ntruss.com',
    api_key='your_api_key_here',
    api_key_primary_val='your_primary_api_key_here',
    request_id='your_request_id_here'
)
# 개인 HyperClovaEmbedding api를 입력

In [ ]:
# 임베딩 생성 요청
try:
    text = "input text"
    embedding = hyper_clova.execute(text)
    print(f"Embedding: {embedding}")
except ValueError as e:
    print(e)
except requests.exceptions.RequestException as e:
    print(f"Request failed: {e}")

Embedding: [-0.9326172, 0.69628906, -0.515625, 0.12683105, -0.29736328, -0.55615234, 0.2388916, 0.15234375, 0.40039062, -0.68359375, 0.29541016, -0.047424316, 0.36669922, -0.81689453, -0.2619629, -0.34375, -0.16601562, 0.36743164, 0.17993164, -0.2705078, -0.80078125, -0.24499512, -0.08093262, 0.34033203, 0.47387695, 0.9682617, 0.2866211, -0.42407227, -0.58154297, -0.20178223, 0.87646484, 0.0181427, -0.7939453, -2.0625, 0.31201172, -0.11657715, -0.12573242, -0.3149414, -1.5751953, 0.27807617, 0.087524414, -0.59716797, 0.55615234, -0.022842407, -0.27807617, -0.081970215, 1.0087891, -0.39575195, -0.7050781, -0.5683594, 0.054107666, 0.52685547, 1.8037109, -0.1784668, -0.31225586, -0.5307617, -0.2388916, 0.16149902, -1.6572266, -0.07635498, -0.2788086, -0.54296875, -0.48706055, -0.27929688, -0.21374512, 1.7119141, 0.91308594, 0.13464355, -0.703125, -0.67089844, -0.33496094, 0.7866211, -0.3413086, -0.08496094, -1.2548828, 1.1201172, 0.6401367, -0.40893555, -0.41479492, 0.6435547, 0.6928711, 

In [ ]:
# HyperClovaEmbedding 인스턴스와 함께 사용하는 임베딩 함수
embedding_func = HyperClovaEmbeddings(hyper_clova)

In [ ]:
db_law_path = "/content/drive/MyDrive/miraeasset/ChromaDB/DB_law"
db_law = Chroma(collection_name="summaries",
                     embedding_function=embedding_func,
                     persist_directory=db_law_path)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


In [ ]:
# The storage layer for the parent documents
store = InMemoryStore()
id_key = "doc_id"

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=db_law,
    docstore=store,
    id_key=id_key,
)

# Add texts
doc_ids = [str(uuid.uuid4()) for _ in texts]
summary_texts = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(text_summaries)
]
retriever.vectorstore.add_documents(summary_texts)
retriever.docstore.mset(list(zip(doc_ids, texts)))

# Add tables
table_ids = [str(uuid.uuid4()) for _ in tables]
summary_tables = [
    Document(page_content=s, metadata={id_key: table_ids[i]})
    for i, s in enumerate(table_summaries)
]
retriever.vectorstore.add_documents(summary_tables)
retriever.docstore.mset(list(zip(table_ids, tables)))

Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 20 seconds...


In [ ]:
retriever.vectorstore.similarity_search_with_score("세액공제 ") # 확인용

[(Document(metadata={'doc_id': 'bdb7102a-e0be-469a-963c-23512e9eafa1'}, page_content='- 전자세금계산서 발급 세액공제 적용대상을 확대함'),
  497.0826416015625),
 (Document(metadata={'doc_id': '0238037d-8183-49ea-bc27-6b996257277a'}, page_content='- 과세 형평 제고를 위해 조특법상 세액공제의 소득기준 및 월세액 한도 상향함'),
  498.9779968261719),
 (Document(metadata={'doc_id': 'd4b42141-e30e-48e9-a517-888072041099'}, page_content='- 전자계산서 발급 세액공제 적용대상 확대에 대해 이야기하고 있음'),
  514.1574096679688),
 (Document(metadata={'doc_id': 'c6024cca-3007-4e44-a6c3-c3ca3a7a89e7'}, page_content='- 세액공제 대상 납입한도 세액 공제율을 설명하고 있음'),
  515.3011474609375)]

In [ ]:
retriever.vectorstore.similarity_search_with_score("주택청약종합저축") # 확인용

[(Document(metadata={'doc_id': '8335ad16-d481-43c0-86f2-a38106637334'}, page_content='- 주택청약종합저축 납입액에 대한 소득공제 공제대상 납입한도 상향함\n- 무주택 세대주, 총급여액 7,000만원 이하인 근로소득자에게 납입액의 40%를 근로소득금액에서 공제해 줌\n- 적용기한은 ’25.12.31.임'),
  420.3085021972656),
 (Document(metadata={'doc_id': '2a5c27f4-8790-4c6d-acb7-e53b4d2d7575'}, page_content='- 민간의 벤처투자를 지원하기 위해 과세표준을 신고하거나 연말정산하는 분부터 적용함\n- 주택청약종합저축 소득공제 한도를 상향함'),
  610.1844482421875),
 (Document(metadata={'doc_id': '22238dc9-900a-4513-a289-8ac4f0ed8e99'}, page_content='- 고령층 노후주거 안정 및 소득확보 지원을 위해 장기주택저당차입금 이자상환액 소득공제 확대 등을 개정함'),
  626.4319458007812),
 (Document(metadata={'doc_id': 'e8b06a32-0212-41e2-9f0c-30648d9c2198'}, page_content='- 장기 주택저당 차입금 이자상환액 소득공제 대상을 확대함'),
  639.7756958007812)]

In [ ]:
# vectorstore 저장
db_law.persist()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [ ]:
# docstore 저장
with open("/content/drive/MyDrive/miraeasset/ChromaDB/DB_law/db_law_docstore.pkl", "wb") as f:
    pickle.dump(store, f)

# db_miraeasset

In [ ]:
path = "/content/drive/MyDrive/miraeasset/data/miraeasset"
config = PDFPartitionConfig(image_output_dir_path=path)

In [ ]:
raw_pdf_elements = process_pdfs_in_directory(path, config) # 약 30분 소요(CPU)

In [ ]:
len(raw_pdf_elements)

1

In [ ]:
all_pdf_elements = []
for _, elements in raw_pdf_elements:
    all_pdf_elements.extend(elements)

In [ ]:
# Create a dictionary to store counts of each type
category_counts = {}

for element in all_pdf_elements:
    category = str(type(element))
    if category in category_counts:
        category_counts[category] += 1
    else:
        category_counts[category] = 1

# Unique_categories will have unique elements
unique_categories = set(category_counts.keys())
category_counts

{"<class 'unstructured.documents.elements.CompositeElement'>": 157,
 "<class 'unstructured.documents.elements.Table'>": 131,
 "<class 'unstructured.documents.elements.TableChunk'>": 4}

In [ ]:
class Element(BaseModel):
    type: str
    text: Any


# Categorize by type
categorized_elements = []
for element in all_pdf_elements:
    if "unstructured.documents.elements.Table" in str(type(element)):
        categorized_elements.append(Element(type="table", text=str(element)))
    elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
        categorized_elements.append(Element(type="text", text=str(element)))

# Tables
table_elements = [e for e in categorized_elements if e.type == "table"]
print(len(table_elements))

# Text
text_elements = [e for e in categorized_elements if e.type == "text"]
print(len(text_elements))

135
157


## Summaries

In [ ]:
tables = [i.text for i in table_elements]
table_summaries = process_texts_concurrently(tables) # 약 7분 소요(CPU)

Error: HTTP error: 400 - Bad Request
Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 40 seconds...
Rate limit exceeded. Retrying in 40 seconds...
Rate limit exceeded. Retrying

In [ ]:
len(table_summaries)

135

In [ ]:
texts = [i.text for i in text_elements]
text_summaries = process_texts_concurrently(texts)

Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 40 seconds...
Rate limit exceeded. Retrying in 40 seconds...
Rate limit exceeded. Retrying in 40 seconds...
Rate limit exceeded

In [ ]:
len(text_summaries)

157

In [ ]:
tables[10:13] # 확인용

['미래에셋멀티솔루션일반사모부동산투자신탁1호 아르게스-AFWP 스마트모빌리티 신기술투자조합 제1호 엔켐 11CB 에코프로비엠 1CB KG모빌리티 122BW 투자분기 3Q23 1Q23 2Q23 2Q23 4Q23 투자금액 200 30 30 30 27 ESG구분 Sustainability Green Green Green Green',
 '발행분기 2Q23 1Q23 1Q23 2Q23 2Q23 주관·인수 중 Max규모(중복제외) 5,050 3,200 2,500 2,000 2,000 ESG구분 Green Green Green Green Green',
 '연말기준 4Q23 4Q23 4Q23 4Q23 4Q23 장부수량 1,000 1,000 800 700 586 ESG구분 Sustainability Social Social Social Social']

In [ ]:
table_summaries[10:13] # 확인용

['- 투자한 기업들의 ESG 구분을 하고 있음',
 '- 주관·인수 중 Max규모는 5,050임\n- ESG 구분은 Green임',
 '- 4분기 기준 장부 수량을 이야기하고 있음']

## Add to vectorstore

In [ ]:
db_miraeasset_path = "/content/drive/MyDrive/miraeasset/ChromaDB/DB_miraeasset"
db_miraeasset = Chroma(collection_name="summaries",
                     embedding_function=embedding_func,
                     persist_directory=db_miraeasset_path)

In [ ]:
# The storage layer for the parent documents
store = InMemoryStore()
id_key = "doc_id"

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=db_miraeasset,
    docstore=store,
    id_key=id_key,
)

# Add texts
doc_ids = [str(uuid.uuid4()) for _ in texts]
summary_texts = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(text_summaries)
]
retriever.vectorstore.add_documents(summary_texts)
retriever.docstore.mset(list(zip(doc_ids, texts)))

# Add tables
table_ids = [str(uuid.uuid4()) for _ in tables]
summary_tables = [
    Document(page_content=s, metadata={id_key: table_ids[i]})
    for i, s in enumerate(table_summaries)
]
retriever.vectorstore.add_documents(summary_tables)
retriever.docstore.mset(list(zip(table_ids, tables)))

Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 20 seconds...


In [ ]:
retriever.vectorstore.similarity_search_with_score("미래에셋 지속경영") # 확인용

[(Document(metadata={'doc_id': '193478e2-f5cc-4f0d-9f74-19cddfba14aa'}, page_content='- 미래에셋증권은 ESG 경영을 위해 지속가능경영을 추진해 나갈 것임\n- 미래에셋증권은 고객의 평안한 노후 제공과 건강한 사회 구현을 위해 UN 지속가능 발전목표(UN SDGs)를 지원하는 지속가능금융을 집계하고 있음\n- 2025년까지 지속가능금융 45조 원 달성 목표를 수립하여 지난해 말 기준, 약 34.4조 원의 지속가능금융을 제공함\n- 환경사회정책 선언문(ESP)은 비재무적 리스크 관리 기준과 이행 프로세스를 제시하고 있음\n- 미래에셋증권은 고객과 함께 사회적 책임을 다하고 지속가능한 성장을 시현할 수 있는 자문과 서비스, 그리고 양질의 금융상품을 선별적으로 제공할 것임\n- 미래에셋증권은 과학에 기반한 온실가스 감축목표 기준을 제시하는 SBTi를 통해 온실가스 감축 목표에 대한 검증을 완료함\n- 자산군별 감축 목표 이행현황을 지속적으로 모니터링하며 지구온난화 이슈 대응에 동참할 것임\n- 금융취약계층을 대상으로 교육 및 서비스 등을 제공하고 포용적 성장을 위한 사회적 역할을 성실히 실천하고 있음\n- 미래에셋박현주재단을 중심으로 계열사 간 협업을 통해 장학과 사회복지 사업을 수행하며 지역사회와 동반성장 할 수 있는 사회공헌 활동을 꾸준히 전개하고 있음'),
  463.9093017578125),
 (Document(metadata={'doc_id': '5a235fbe-282b-4e2b-878f-45b395ff3c0e'}, page_content='- 미래에셋증권은 2025년까지 지속가능금융 45조 원 달성을 목표로 설정함\n- 금융배출량 지표를 기반으로 과학기반 감축목표를 설정함\n- VPPA계약 및 REC장기구매계약을 통해 2023년 말 기준 34.4조 원 달성 설정함\n- SBTi 검증을 확보 체결을 통해 2024년 4월 계약 누적 건수를 설정함\n- 경영진 및 조직성과 KPI에 지속가능경

In [ ]:
retriever.vectorstore.similarity_search_with_score("예탁유가증권") # 확인용

[(Document(metadata={'doc_id': 'fbecdfdb-64b7-4f98-9107-e2d54fb35d34'}, page_content=''),
  694.5873938955733),
 (Document(metadata={'doc_id': '1d1c0c5e-2772-4170-bf1e-79ae808e6d51'}, page_content=''),
  694.5873938955733),
 (Document(metadata={'doc_id': '8cc6af49-976a-4456-ab2e-49d6afcb0080'}, page_content=''),
  694.5873938955733),
 (Document(metadata={'doc_id': '67846474-9c67-44f0-9672-ed4d62ddc814'}, page_content=''),
  694.5873938955733)]

In [ ]:
retriever.vectorstore.similarity_search_with_score("sk하이닉스") # 확인용,

[(Document(metadata={'doc_id': '0cdd76f1-eb14-40d9-898a-7c7a24e9c77a'}, page_content=''),
  686.3842143176731),
 (Document(metadata={'doc_id': 'fbecdfdb-64b7-4f98-9107-e2d54fb35d34'}, page_content=''),
  686.3842143176731),
 (Document(metadata={'doc_id': 'ad1ab0d9-60d4-4b41-bd00-91af77b8d86e'}, page_content=''),
  686.3842143176731),
 (Document(metadata={'doc_id': '67846474-9c67-44f0-9672-ed4d62ddc814'}, page_content=''),
  686.3842143176731)]

In [ ]:
retriever.vectorstore.similarity_search_with_score("주택청약종합저축") # 확인용

[(Document(metadata={'doc_id': '1d1c0c5e-2772-4170-bf1e-79ae808e6d51'}, page_content=''),
  658.7703326143584),
 (Document(metadata={'doc_id': 'fbecdfdb-64b7-4f98-9107-e2d54fb35d34'}, page_content=''),
  658.7703326143584),
 (Document(metadata={'doc_id': '8cc6af49-976a-4456-ab2e-49d6afcb0080'}, page_content=''),
  658.7703326143584),
 (Document(metadata={'doc_id': '0cdd76f1-eb14-40d9-898a-7c7a24e9c77a'}, page_content=''),
  658.7703326143584)]

In [ ]:
# vectorstore 저장
db_miraeasset.persist()

In [ ]:
# docstore 저장
with open("/content/drive/MyDrive/miraeasset/ChromaDB/DB_miraeasset/db_miraeasset_docstore.pkl", "wb") as f:
    pickle.dump(store, f)

# db_economy

In [ ]:
path = "/content/drive/MyDrive/miraeasset/data/economy"
config = PDFPartitionConfig(image_output_dir_path=path)

In [ ]:
raw_pdf_elements = process_pdfs_in_directory(path, config)

In [ ]:
len(raw_pdf_elements)

3

In [ ]:
all_pdf_elements = []
for _, elements in raw_pdf_elements:
    all_pdf_elements.extend(elements)

In [ ]:
# Create a dictionary to store counts of each type
category_counts = {}

for element in all_pdf_elements:
    category = str(type(element))
    if category in category_counts:
        category_counts[category] += 1
    else:
        category_counts[category] = 1

# Unique_categories will have unique elements
unique_categories = set(category_counts.keys())
category_counts

{"<class 'unstructured.documents.elements.CompositeElement'>": 116,
 "<class 'unstructured.documents.elements.Table'>": 68}

In [ ]:
class Element(BaseModel):
    type: str
    text: Any


# Categorize by type
categorized_elements = []
for element in all_pdf_elements:
    if "unstructured.documents.elements.Table" in str(type(element)):
        categorized_elements.append(Element(type="table", text=str(element)))
    elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
        categorized_elements.append(Element(type="text", text=str(element)))

# Tables
table_elements = [e for e in categorized_elements if e.type == "table"]
print(len(table_elements))

# Text
text_elements = [e for e in categorized_elements if e.type == "text"]
print(len(text_elements))

68
116


## Summaries

In [ ]:
tables = [i.text for i in table_elements]
table_summaries = process_texts_concurrently(tables)

Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 40 seconds...
Rate limit exceeded. Retrying in 40 seconds...
Rate limit exceeded. Retrying in 40 seconds...
Rate limit exceeded

In [ ]:
len(table_summaries)

68

In [ ]:
texts = [i.text for i in text_elements]
text_summaries = process_texts_concurrently(texts)

Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 40 seconds...
Rate limit exceeded. Retrying in 40 seconds...
Rate limit exceeded. Retrying in 40 seconds...
Rate limit exceeded

In [ ]:
len(text_summaries)

116

In [ ]:
tables[10:13] # 확인용

['구분 특징 보험료 부담 당연 (의무) 가입 대상 사업장 가입자 지역 가입자 근로자 1인 이상 사업장에 종사하는 18세 이상 60세 미 만 근로자와 사용자 18세 이상 60세 미만인 자로서 사업장 가 입자가 아니면 서 다른 공적연금(공무원연금 등)의 가입자가 아닌 자 소득의 9% (근로자 4.5% + 사용자 4.5%) 소득의 9% 임의 가입 대상 임의 가입자 임의 계속 가입자 사업장 및 지역 가입자가 아니면서 60세 이전에 가입 한 자 60세 이상 ~ 65세 미만인 자로서 신청하여 가입한 자 9만 원 ~ 45만 2,700원 가입유형별 상이 (사업장/지역/임의)',
 '유형 DB형 DC형 IRP 퇴직급여 수준 계속 근로기간 1년당 30일 분의 평균임금×근속연수 (연봉의 1/12[8.33%] ×근속연수) + 운용손익 퇴직급여 + 본인부담금 + 운용손익 중도인출 불가 예외적인 경우 가능 담보대출 가능 (적립금의 최대 50% 수준) 유리한 경우 •승진기회 높음 • 급여인상률이 (투자)수 익률보다 높음 • 재무구조가 안정적인 •승진기회 적음 • 급여인상률이 (투자) 수익률보다 낮음 • 재무구조가 불안정한 - 기업 기업 •임금피크제',
 '연금저축신탁 연금저축펀드 연금저축보험 판매 금융회사 은행 증권회사, 은행, 보험회사 생명보험사, 손해보험사 (은행 등도 판매) 납입방식 자유적립식 자유적립식 정기납입 적용금리 (수익률) 실적배당 (원금 비보장*) 실적배당 (원금 비보장) 공시이율 (원금 보장) 비용 신탁보수 (적립금에 비례) 보수 (적립금에 비례) 납입금(보험료)의 일부 연금수령방식 확정기간형 확정기간형 확정기간형, 종신형(생명보험사 상품에 한함) 예금자보호 보호 비보호 보호 기타 신규 가입 불가 연금저축계좌 안에서 다수의 펀드 및 ETF 투자 가능 납입유예제도']

In [ ]:
table_summaries[10:13] # 확인용

['- 보험료 부담은 당연 가입 대상 사업장 가입자, 지역 가입자, 근로자 1인 이상 사업장에 종사하는 18세 이상 60세 미만 근로자와 사용자, 임의 가입자, 임의 계속 가입자로 나뉨',
 '- DB형, DC형, IRP 퇴직급여 수준에 대해 설명하고 있음',
 '- 연금저축신탁, 연금저축펀드, 연금저축보험 판매 금융회사를 설명하고 있음']

## Add to vectorstore

In [ ]:
db_economy_path = "/content/drive/MyDrive/miraeasset/ChromaDB/DB_economy"
db_economy = Chroma(collection_name="summaries",
                     embedding_function=embedding_func,
                     persist_directory=db_economy_path)

In [ ]:
# The storage layer for the parent documents
store = InMemoryStore()
id_key = "doc_id"

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=db_economy,
    docstore=store,
    id_key=id_key,
)

# Add texts
doc_ids = [str(uuid.uuid4()) for _ in texts]
summary_texts = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(text_summaries)
]
retriever.vectorstore.add_documents(summary_texts)
retriever.docstore.mset(list(zip(doc_ids, texts)))

# Add tables
table_ids = [str(uuid.uuid4()) for _ in tables]
summary_tables = [
    Document(page_content=s, metadata={id_key: table_ids[i]})
    for i, s in enumerate(table_summaries)
]
retriever.vectorstore.add_documents(summary_tables)
retriever.docstore.mset(list(zip(table_ids, tables)))

Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 5 seconds...
Rate limit exceeded. Retrying in 10 seconds...
Rate limit exceeded. Retrying in 20 seconds...


In [ ]:
retriever.vectorstore.similarity_search_with_score("미래에셋 지속경영") # 확인용

[(Document(metadata={'doc_id': 'ce8a7512-9776-41ed-ae2c-a84097651097'}, page_content='- 정부는 매년 다양한 복지서비스를 늘려 국민들의 노후생활 안정을 지원하고 있음\n- 복지정책과 서비스에 대한 정보가 부족해 혜택을 받지 못하는 경우가 있음\n- 이럴 땐 보건복지부의 복지로를 활용해 노후 관련 복지서비스를 찾아보는 것이 좋음\n- 치매국가책임제를 통해 장기요양서비스를 이용한 A 씨의 사연을 소개하고 있음\n- 보건복지부에 따르면 2030년 치매인구는 전체 노인인구의 10%로 약 127만 명이 될 것으로 예상됨\n- 치매국가책임제는 전국 256개 보건소에 설치된 치매안심센터에서 치매어르신과 가족들은 1:1 맞춤형 상담과 검진, 관리 서비스 연결 등 통합적 지원을 받을 수 있음\n- 센터 내부에 설치된 치매 단기 쉼터와 치매 카페는 어르신의 건강 안정화와 치매 지연을 돕고 정서적 지지 기반을 제공함\n- 센터에서 받은 상담 내용은 ‘치매노인등록관리시스템’을 통해 관리됨\n- 중앙치매센터는 웹사이트(***.***.**.**)를 통해 치매에 관한 기사를 한눈에 볼 수 있게 제공함\n- 치매자가진단 서비스, 전문가 상담을 무료로 받을 수 있음\n- 기초연금은 국민의 안정된 노후생활을 위해 도입됨\n- 2007년 기초연금 제도를 도입함\n- 국민연금과 기초연금을 더해 더 많은 사람들이 혜택을 받게 됨\n- 기초연금 수급 대상자는 대한민국 국적의 만 65세 이상으로 국내에 거주하고 있어 2) 보건복지부, 「2015년 기준 치매환자 의료비 및 요양비」 ,1인당 연간 총 2,033만 원 SUF7/S GALLS &A7| 자산 관리 역량 위기 관리 역량 변화 대응 역량 노후 대비 역량 야 함'),
  808.5718383789062),
 (Document(metadata={'doc_id': '4563c4b0-9864-46f9-8d9f-7b5af9d17ba6'}, page_content='- 노후 생활비를 계산하

In [ ]:
retriever.vectorstore.similarity_search_with_score("예탁유가증권") # 확인용

[(Document(metadata={'doc_id': '2f930478-8763-4865-8959-5ace7a348a50'}, page_content='- 신탁형 펀드형 보험형 운용기관은 은행, 자산운용사, 보험사임\n- 이익배당은 실적 배당임\n- 상품 종류는 채권형, 안정형, 주식형임'),
  742.4388466661691),
 (Document(metadata={'doc_id': 'e95b4468-b71b-4cd0-a442-31cffcd608db'}, page_content='- 투자성향별 점수표를 보여주고 있음'),
  754.1248779296875),
 (Document(metadata={'doc_id': 'fb747561-0f70-41bb-bbcc-88583fe826bd'}, page_content='- 투자성향 점수를 알려주고 있음'),
  755.7100601178588),
 (Document(metadata={'doc_id': '42d13720-7bd8-4a9b-a3c6-5e5aa1b9ba7f'}, page_content='- 연금저축신탁, 연금저축펀드, 연금저축보험 판매 금융회사를 설명하고 있음'),
  803.3542371058641)]

In [ ]:
retriever.vectorstore.similarity_search_with_score("주택청약종합저축") # 확인용

[(Document(metadata={'doc_id': '9acbb8a1-b029-4273-98af-a45a6dd5dc49'}, page_content='- 청년우대형 주택청약종합저축은 만 19~34세의 무주택세대주, 무주택세대주 예정자, 무주택 세대원이 가입할 수 있음\n- 납입원금은 5,000만 원 한도임\n- 이자율은 1개월 이내 1개월 초과∼ 1년 미만 1년 이상 ∼ 2년 미만 2년 이상∼ 10년 이내 10년 초과 시 무이자 연 2.5% 연 3.0% 연 3.3% 연 1.8%임\n- 청약 1순위 자격은 주택의 유형에 따라 다름\n- 전세로 독립할 경우, 월세보다 주거 비용을 크게 낮출 수 있음\n- 전세를 구하기 힘들다는 조급함을 악용한 전세 사기가 빈발하고 있으므로, 전세 계약 전후로 주의해야 할 필요가 있음\n- 전세보증금을 지키기 위해 해야 할 일을 설명하고 있음'),
  472.043212890625),
 (Document(metadata={'doc_id': 'b14937ac-6da0-407a-91aa-4753760220ba'}, page_content='- 연금저축을 옮기고자 하는 금융회사에서 연금저축 계좌이체를 신청한 후, 전화 또는 지점방문으로 해당 신청 사실을 확인해주면 됨\n- 다양한 연금상품 비교 후 자신에게 가장 유리한 상품을 선택하길 바람\n- 통합연금포털에서 퇴직연금과 연금저축을 한 자리에서 비교할 수 있음\n- 연금저축 비교공시에서 원리금보장 연금상품, 퇴직연금 맞춤형 수수료 비교를 할 수 있음\n- 주택 구입에 수억 원 이상 드는 만큼, 지금부터 내 집을 마련하기 위해 노력해야 함\n- 내 집 마련의 첫 단계는 주택청약종합저축 계좌를 개설하는 것임\n- 청년 우대형 주택청약종합저축은 신규 아파트의 청약 자격을 부여하는 통장임\n- 오래, 꾸준히 적립하며 유지할수록 청약 시 유리하니 지금부터 활용할 필요가 있음'),
  561.7218017578125),
 (Document(metadata={'doc_id': 'f96beabd-6

In [ ]:
retriever.vectorstore.similarity_search_with_score("장년층 노후대비") # 확인용

[(Document(metadata={'doc_id': '4563c4b0-9864-46f9-8d9f-7b5af9d17ba6'}, page_content='- 노후 생활비를 계산하고 있음'),
  448.164412383352),
 (Document(metadata={'doc_id': 'fb4134f9-0f0b-4a18-81aa-5e8e119b3afa'}, page_content='- 노후 대비 역량을 보충할 것인지 고민해야 함\n- 수입을 늘리는 방법에는 취업, 보유 자산의 적극적인 운용 등의 방법이 있음\n- 은퇴 후 줄어든 소득을 토대로 새로운 계획을 세우고 실천해야 함\n- 노후 경제활동을 위한 팁은 경제 자립도를 높이는 것이 우선임\n- 노년층을 위한 정부 정책을 적극 활용함\n- 노년층을 위한 정부의 각종 지원제도를 알아보고 적극적으로 활용함\n- 배우는 삶을 즐김\n- 주민센터와 같은 기관에 많은 강좌가 개설되어 있음\n- 배움과 함께 이웃들과 교류하면서 즐거움과 행복감을 느낄 수 있음\n- 노후를 위한 각종 복지 제도를 이해하고 활용할 수 있도록 매스컴이나 주민센터를 통해 교육이 필요함\n- 주문을 잊은 음식점은 치매 환자와 가족들에게 국한된 문제가 아닌 우리 사회의 문제라는 관점으로 변화를 이끌어낸 점에서 시청자의 호평을 받음\n- 노후에 힘이 되는 복지서비스 중에 연금과 생활지원 분야를 중심으로 살펴 보겠음'),
  515.076904296875),
 (Document(metadata={'doc_id': 'a7092fa3-98ff-4bfb-9d44-6566e06c35c7'}, page_content='- 노후 준비가 필요한 이유를 설명하고 있음\n- 노후 준비를 위해 3층 연금 체계를 갖추는 게 중요함\n- 국민의 노후 보장을 위한 공적연금인 국민연금, 기업 차원의 퇴직연금, 개인이 추가적으로 가입하는 개인연금으로 구성됨\n- 국민연금은 노후에 수입이 정기적으로 발생하지 않거나 재산이 없는 경우를 대비하기 위한 안전장치임\n- 국민연금 수령액을

In [ ]:
retriever.vectorstore.similarity_search_with_score("적극투자성향") # 확인용

[(Document(metadata={'doc_id': 'fb747561-0f70-41bb-bbcc-88583fe826bd'}, page_content='- 투자성향 점수를 알려주고 있음'),
  487.1682463904996),
 (Document(metadata={'doc_id': 'e95b4468-b71b-4cd0-a442-31cffcd608db'}, page_content='- 투자성향별 점수표를 보여주고 있음'),
  519.5343017578125),
 (Document(metadata={'doc_id': 'd281c799-3e7f-4395-9074-c6c3bf6eaefc'}, page_content='- 안정추구형 투자는 원금의 손실위험은 최소화하고 이자 소득이나 배당소득 수준의 안정적인 투자를 목표로 함\n- 채권형펀드가 적당함\n- 위험중립형 투자에는 그에 상응하는 투자위험이 있음을 충분히 인식하고 있음\n- 예·적금보다 높은 수익을 기대할 수 있다면 일정수준의 손실위험을 감수할 수 있음\n- 적립식펀드나 주가연동상품처럼 중위험 펀드로 분류되는 상품을 선택하는 것이 좋음\n- 적극투자형 투자원금의 보전보다는 위험을 감내하더 라도 높은 수준의 투자수익을 추구함\n- 공격투자형 시장평균수익률을 훨씬 넘어서는 높은 수준의 투자수익을 추구함'),
  566.0608481883963),
 (Document(metadata={'doc_id': '21ea503b-4356-4ee8-baa4-dcad0521a26f'}, page_content='- 투자성향 등급 유형의 의미를 설명하고 있음'),
  606.4522815651004)]

In [ ]:
# vectorstore 저장
db_economy.persist()

In [ ]:
# docstore 저장
with open("/content/drive/MyDrive/miraeasset/ChromaDB/DB_economy/db_economy_docstore.pkl", "wb") as f:
    pickle.dump(store, f)

# db_company
- save_DB_company_ver.ipynb에서 실행 완료